In [ ]:
!pip install kora -q
from kora import drive
drive.link_nbs()

In [ ]:
import utils
import ResNet18
import OF_CNN

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from keras.applications.efficientnet import EfficientNetB0
from keras.models import Model, Sequential
from keras.layers import TimeDistributed, Input, MaxPooling3D, LSTM, Dense, Conv3D, Conv2D, BatchNormalization, Flatten, Dropout, Convolution2D, Activation, MaxPooling2D, GlobalAveragePooling2D
from keras import losses

In [ ]:
""" GLOBAL VARIABLES """
NUM_CLASSES = 4
EPOCHS = 10
INSTACK = 16
IMG_SIZE = (224, 224)
LEN_TRAIN = 85

In [ ]:
""" Data loading """

train_gen = joinedGen(Train=True)
val_gen = joinedGen(Val=True)

In [ ]:
TVHI_weights = "/___/____/"
OF_weights = "___/___/_"

In [ ]:
""" Building Model """

#Baseline model to keep expanding based on example from: https://keras.io/guides/training_with_built_in_methods/#passing-data-to-multiinput-multioutput-models
def build_TS_model():
  img_input = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="img_input")
  of_input = keras.Input(shape=(INSTACK,IMG_SIZE[0],IMG_SIZE[1],3), name="of_input")

  #Set up basemodels, load weights, remove certain layers, check output sizes. If not the same, use 1D convolutions, fuse together dense layers using the concatenate. 
  #Freeze baseline models. Set new layers to trainable. Train. Save weights and model. 
  resnet_base = ResNet18.ResNet18(NUM_CLASSES)
  resnet_base.load_weights(TVHI_weights)

  of_base = OF_CNN.build_OF_model((INSTACK, IMG_SIZE[0], IMG_SIZE[1], 3), NUM_CLASSES)
  of_base.load_weights(OF_weights)

  x1 = layers.Conv2D(3, 3)(image_input)
  x1 = layers.GlobalMaxPooling2D()(x1)

  x2 = layers.Conv1D(3, 3)(timeseries_input)
  x2 = layers.GlobalMaxPooling1D()(x2)

  x = layers.concatenate([x1, x2])


  class_output = layers.Dense(NUM_CLASSES,  activation='softmax', name="pred")(x)

  model = keras.Model(
      inputs=[image_input, timeseries_input], outputs=[score_output, class_output]
  )
  model.compile(loss='categorical crossentropy', optimizer='Adam', metrics=['accuracy'])
  model.summary()

  return model 


In [ ]:
""" Plot the model """
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
""" Set up callback and perform training """
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

ts_history = of_model.fit(train_gen,epochs= EPOCHS, steps_per_epoch = LEN_TRAIN, validation_data=val_gen, validation_steps=15, callbacks= callback, verbose=1)

In [ ]:
""" Save model and weights """
ts_model.save_weights('TS_weights.h5')
ts_model.save("TS_model", save_format = 'tf', include_optimizer=True)